In [1]:
%matplotlib inline

import pandas as pd
import stumpy
import numpy as np
import datetime as dt
import random
import math
import pickle
from statistics import mean
from tqdm.auto import tqdm
import sys

import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [2]:
with open('../nonzero_traces.npy', 'rb') as f:
    traces = pickle.load(f)

In [3]:
def merge_traces(trace_1, id_2, num_traces):
    trace2 = random.sample(traces[id_2], 5)
    trace2_flat = np.asarray([item for row in trace2 for item in row]).astype('float64')
    return trace2_flat

In [4]:
'''
Input: 
    trace1: packet traces from class 1
    id2: id number for class 2 
    num_traces: number of traces to select from class 2 (should be equal to class 1)
    shapelet_size: length of shapelets
    
Output:
    Matrix profile of trace1 with itself and trace1 with trace2
'''
def generate_profile(trace1, id2, num_traces, shapelet_size):
    
    trace2 = []
    while(len(trace2) < shapelet_size):
        trace2 = random.sample(traces[id2], num_shapelets)
        trace2 = np.asarray([item for row in trace2 for item in row]).astype('float64')
    
    c1_c1 = stumpy.stump(trace1, shapelet_size)[:, 0].astype(float)
    c1_c1[c1_c1 == np.inf] = np.nan
    
    #print("self-profiles generated...")
    c1_c2 = stumpy.stump(trace1, shapelet_size, trace2, ignore_trivial=False)[:, 0].astype(float)
    c1_c2[c1_c2 == np.inf] = np.nan
    #print("Comparison profiles generated...")
    
    return c1_c1, c1_c2

def generate_shapelet(trace, diff, shapelet_size):
    
    idx = np.argmax(diff)
    shapelet = np.asarray([1 if idx <= i < idx + shapelet_size else 0 for i in range(len(trace))])
    
    return shapelet

In [8]:
def find_overlap(trace_i, shapelets_i, shapelet_size):
    #print(shapelets_i[0])
    
    merged_shapelets = np.sum(shapelets_i, axis=0)
    
    max_size = 0
    start = 0
    end = 0
    
    for i in range(0, len(merged_shapelets), shapelet_size):
        current_size = np.sum(merged_shapelets[i:i+shapelet_size])
        if current_size > max_size:
            max_size = current_size
            start = i
            end = i + shapelet_size
    
    return trace_i[start:end]

In [12]:
shapelet_storage = []
shapelet_size=400
num_traces = 5

for i in tqdm(range(100)):
    # get packet traces for i and all others
    
    trace_i = []
    while(len(trace_i) < shapelet_size):
        trace_i = random.sample(traces[i], num_traces)
        trace_i = np.asarray([item for row in trace_i for item in row]).astype('float64')
        
    
    shapelets_i = np.zeros((100, len(trace_i)))
    #print(shapelets_i.shape)
    
    for j in tqdm(range(100)):
        if i == j:
            continue
        
        # compute differences
        ci_ci, ci_cj = generate_profile(trace_i, j, shapelet_size)

        # find largest value gap between self and other
        diff_ci = ci_cj - ci_ci
        
        ci_shape = generate_shapelet(trace_i, diff_ci, shapelet_size)
        
        shapelets_i[j] = ci_shape
    
    best_shapelet = find_overlap(trace_i, shapelets_i, shapelet_size)
    shapelet_storage.append(best_shapelet)


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

9172.0
[ 2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  3.
  4.  4.  4. 24. 26. 26. 26. 26. 26. 26. 26. 26. 26. 26. 26. 26. 26. 25.
 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24.
 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24.
 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24.
 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24.
 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24.
 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24.
 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24.
 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24.
 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24.
 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24.
 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24.
 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24.

  0%|          | 0/100 [00:00<?, ?it/s]

12975.0
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.
  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  4.  4.  7.  7.
  7.  8.  9.  9. 12. 13. 13. 17. 17. 17. 17. 17. 17. 49. 52. 52. 52. 52.
 52. 53. 55. 55. 56. 56. 56. 56. 56. 56. 56. 56. 56. 56. 56. 56. 56. 56.
 56. 56. 56. 56. 56. 56. 56. 56. 56. 56. 56. 56. 56. 56. 56. 56. 56. 56.
 56. 56. 56. 56. 56. 56. 56. 56. 56. 56. 56. 56. 56. 56. 56. 56. 56. 56.
 56. 56. 56. 56. 56. 56. 56. 56. 56. 56. 56

...

16929.0
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  1.  9. 10. 11. 11. 11. 11.
 11. 11. 11. 11. 11. 11. 11. 14. 15. 15. 17. 17. 17. 17. 17. 17. 17. 17.
 17. 54. 54. 55. 55. 55. 55. 56. 57. 58. 58. 61. 61. 61. 61. 62. 63. 64.
 64. 64. 69. 69. 69. 69. 70. 70. 71. 71. 71. 71. 71. 71. 71. 71. 71. 71.
 71. 71. 71. 71. 71. 71. 71. 71. 71. 71. 71. 71. 71. 71. 71. 71. 71. 71.
 71. 71. 71. 71. 71. 71. 71. 71. 71. 71. 71. 71. 71. 71. 71. 71. 71. 71.
 71. 71. 71. 71. 71. 71. 71. 71. 71. 71. 71

  0%|          | 0/100 [00:00<?, ?it/s]

34452.0
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 99. 99.
 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99.
 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99.
 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99.
 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99.
 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99.
 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99.
 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99.
 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99.
 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99.
 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99.
 99. 99. 99. 99. 99. 99. 99. 99. 99. 99. 99

  0%|          | 0/100 [00:00<?, ?it/s]

20426.0
[66. 66. 66. 66. 66. 66. 66. 66. 66. 66. 66. 66. 66. 66. 66. 66. 66. 66.
 66. 66. 66. 66. 66. 66. 66. 66. 66. 66. 66. 66. 66. 66. 66. 66. 66. 66.
 66. 66. 66. 66. 66. 66. 66. 66. 66. 66. 66. 66. 66. 66. 66. 66. 66. 66.
 66. 66. 66. 66. 66. 66. 66. 66. 66. 66. 66. 66. 66. 66. 66. 66. 66. 66.
 66. 66. 66. 66. 66. 66. 66. 66. 66. 66. 66. 66. 66. 66. 66. 66. 66. 66.
 66. 66. 66. 66. 66. 66. 66. 66. 66. 66. 66. 66. 66. 66. 66. 66. 66. 66.
 66. 66. 66. 66. 66. 66. 66. 66. 66. 66. 66. 66. 66. 66. 66. 66. 66. 66.
 66. 66. 66. 66. 65. 65. 65. 65. 65. 65. 65. 65. 65. 65. 65. 65. 65. 65.
 65. 65. 65. 65. 65. 65. 65. 65. 65. 65. 65. 65. 65. 65. 65. 65. 65. 65.
 65. 65. 65. 65. 65. 64. 64. 64. 64. 64. 64. 64. 64. 64. 64. 64. 64. 64.
 64. 64. 64. 64. 64. 64. 64. 64. 64. 64. 64. 64. 64. 64. 64. 64. 64. 64.
 64. 64. 64. 64. 64. 64. 64. 64. 64. 64. 64. 64. 64. 64. 64. 63. 63. 63.
 63. 63. 63. 63. 63. 63. 63. 62. 62. 62. 62. 62. 61. 61. 61. 61. 61. 61.
 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61

In [13]:
with open('../shapelets.pkl', 'wb') as f:
    mynewlist = pickle.dump(shapelet_storage, f)